In [1]:
from statsbombpy import sb
import pandas as pd
from mplsoccer import VerticalPitch,Pitch
from highlight_text import ax_text, fig_text
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.pyplot as plt
import matplotlib.patheffects as path_effects
import seaborn as sns
from datetime import datetime
import os

In [7]:
def collect_shots_data(league, save_path):
    """
    Pobiera i zapisuje dane o strzałach z wybranej ligi dla sezonu 2015/16 (jedyny z ogólnodostępnymi danymi).
    
    Parameters:
    -----------
    league : str lub list
        Nazwa ligi lub lista lig (np. 'Italy', 'England', 'Spain', 'Germany', 'France')
    save_path: str
        Ścieżka zapisu docelowego pliku
        
    Returns:
    --------
    pd.DataFrame
        DataFrame ze wszystkimi strzałami
    """
    if not os.path.exists(save_path):
        os.makedirs(save_path)
    
    # Obsługa pojedynczej ligi lub listy lig
    if isinstance(league, str):
        leagues_to_process = [league]
    else:
        leagues_to_process = league
    
    all_shots_data = []
    
    # Przetwarzanie każdej ligi osobno
    for current_league in leagues_to_process:
        print(f"\nRozpoczynanie pobierania danych z ligi: {current_league}")
        
        # Pobranie danych o lidze
        free_comps = sb.competitions()
        
        # Filtrowanie wybranej ligi
        league_data = free_comps[(free_comps['season_name']=='2015/2016') & 
                           (free_comps['country_name']==current_league)]
        
        if league_data.empty:
            print(f"Nie znaleziono danych dla ligi {current_league}. Pomijam.")
            continue
        
        competitions = list(league_data['competition_id'])
        
        # Pobranie ID meczów
        try:
            all_matches = pd.concat([sb.matches(competition_id=comp_id, season_id=27) 
                                  for comp_id in competitions])
            matches_id = list(all_matches['match_id'])
            print(f"Znaleziono {len(matches_id)} meczów do przeanalizowania")
        except Exception as e:
            print(f"Błąd podczas pobierania meczów dla ligi {current_league}: {str(e)}")
            continue
        
        # Pobieranie danych o strzałach
        shot_data = []
        
        for idx, match_id in enumerate(matches_id):
            try:
                print(f"Przetwarzam mecz {idx+1}/{len(matches_id)}", end='\r')
                
                events = sb.events(match_id=match_id)
                shots = events[events['type']=='Shot']
                shot_data.append(shots)
                                
            except Exception as e:
                print(f"\nBłąd przy meczu {match_id}: {str(e)}")
                continue
        
        if shot_data:
            # Łączenie danych z tej ligi
            print("\nŁączenie danych...")
            league_shots = pd.concat(shot_data)
            
            # Zapisywanie pliku dla tej ligi
            output_filename = os.path.join(save_path, f'all_shots_{current_league}_2015_2016.csv')
            league_shots.to_csv(output_filename)
            print(f"Zapisano dane do pliku: {output_filename}")
            
            # Dodanie do zbiorczych danych
            all_shots_data.append(league_shots)
    
    # Łączenie danych ze wszystkich lig, jeśli jest więcej niż jedna
    if len(all_shots_data) > 0:
        all_shots = pd.concat(all_shots_data)
        
        # Zapisywanie zbiorczego pliku, jeśli przetworzono więcej niż jedną ligę
        if len(leagues_to_process) > 1:
            output_filename = os.path.join(save_path, f'all_shots_combined_2015_2016.csv')
            all_shots.to_csv(output_filename)
            print(f"\nZapisano zbiorcze dane do pliku: {output_filename}")
        
        return all_shots
    else:
        print("Nie pobrano żadnych danych.")
        return None

In [8]:
shots_df = collect_shots_data(league=['Italy', 'England', 'Spain', 'Germany', 'France'], save_path="D:\\football_project")


Rozpoczynanie pobierania danych z ligi: Italy
Znaleziono 380 meczów do przeanalizowania
Przetwarzam mecz 380/380
Łączenie danych...
Zapisano dane do pliku: D:\football_project\all_shots_Italy_2015_2016.csv

Rozpoczynanie pobierania danych z ligi: England
Znaleziono 380 meczów do przeanalizowania
Przetwarzam mecz 380/380
Łączenie danych...
Zapisano dane do pliku: D:\football_project\all_shots_England_2015_2016.csv

Rozpoczynanie pobierania danych z ligi: Spain
Znaleziono 380 meczów do przeanalizowania
Przetwarzam mecz 380/380
Łączenie danych...
Zapisano dane do pliku: D:\football_project\all_shots_Spain_2015_2016.csv

Rozpoczynanie pobierania danych z ligi: Germany
Znaleziono 306 meczów do przeanalizowania
Przetwarzam mecz 306/306
Łączenie danych...
Zapisano dane do pliku: D:\football_project\all_shots_Germany_2015_2016.csv

Rozpoczynanie pobierania danych z ligi: France
Znaleziono 377 meczów do przeanalizowania
Przetwarzam mecz 377/377
Łączenie danych...
Zapisano dane do pliku: D:\foo